In [1]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import sacrebleu

In [1]:
%%time
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model_name = "/kaggle/input/m2m-bilingual"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

CPU times: user 6.78 s, sys: 1.29 s, total: 8.06 s
Wall time: 12.1 s


In [5]:
df = pd.read_csv("/kaggle/input/mansi-russian-parralel-corpus/overall_80K.csv", index_col=0)

In [6]:
df['to'] = 'mns'

In [7]:
df_other = df.copy()
df_other['to'] = 'ru'
df_other['source'], df_other['target'] = df_other['target'], df_other['source']

# Concatenate the original df and df_other
df = pd.concat([df, df_other], ignore_index=True)

In [8]:
train_df, test_val_df = train_test_split(df, test_size=0.2, random_state=42,shuffle=True, stratify=df['to'])
val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42, shuffle=True, stratify=test_val_df['to'])

In [23]:
%%time
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

CPU times: user 11 ms, sys: 2.87 ms, total: 13.9 ms
Wall time: 12.7 ms


In [17]:
tokenizer.add_special_tokens({'additional_special_tokens': ['<mns_MNS>']})
model.resize_token_embeddings(len(tokenizer))

M2M100ScaledWordEmbedding(128105, 1024, padding_idx=1)

In [18]:
def preprocess_function(examples):
    inputs = tokenizer(examples['source'], truncation=True, padding='max_length', max_length=128)
    targets = tokenizer(examples['target'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

In [24]:
%%time
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

CPU times: user 3.4 s, sys: 4.91 ms, total: 3.4 s
Wall time: 3.4 s


In [12]:
train_dataset = train_dataset.remove_columns(['source', 'target', '__index_level_0__', 'to'])
val_dataset = val_dataset.remove_columns(['source', 'target', '__index_level_0__', 'to'])
test_dataset = test_dataset.remove_columns(['source', 'target', '__index_level_0__', 'to'])

In [13]:
import torch
torch.cuda.empty_cache()

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./mansi_finetuned_biling",
    eval_strategy="no",  # отключаем валидацию 
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4, 
    weight_decay=0.01,
    save_total_limit=5, 
    save_strategy="epoch",  # сохраняем токо после каждой эпохи
    num_train_epochs=1,
    predict_with_generate=True,
    logging_dir="./logs",
    report_to="none", 
    fp16=True, 
    load_best_model_at_end=False,  # вырубаем загрузку лучшей модели в конце
)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # убираем токены pad
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # вычисление метрик
    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])
    chrf = sacrebleu.corpus_chrf(decoded_preds, [decoded_labels])

    return {"bleu": bleu.score, "chrf": chrf.score}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Тренировка модели
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
500,2.130700
1000,0.500800
1500,0.458200
2000,0.428900
2500,0.403100
3000,0.386000
3500,0.378600
4000,0.368300
4500,0.357700
5000,0.352000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


TrainOutput(global_step=8114, training_loss=0.4849136508624778, metrics={'train_runtime': 10487.7495, 'train_samples_per_second': 12.379, 'train_steps_per_second': 0.774, 'total_flos': 3.5167773483073536e+16, 'train_loss': 0.4849136508624778, 'epoch': 0.9999075757108968})

In [17]:
import shutil
from IPython.display import FileLink

shutil.make_archive("m2m_biling_finetune", 'zip', "/kaggle/working/mansi_finetuned_biling/")

'/kaggle/working/m2m_biling_finetune.zip'

In [18]:
FileLink('m2m_biling_finetune.zip')

/kaggle/working/m2m_biling_finetune.zip

In [27]:
%%time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

random.seed(42) 
# random_indices = random.sample(range(len(test_dataset)), 1000)
random_test_samples = test_dataset

def generate_translation(sample):
    input_ids = torch.tensor(sample['input_ids']).unsqueeze(0).to(device)  # Конвертируем список в тензор
    with torch.no_grad():
        generated_ids = model.generate(input_ids, max_length=200, num_beams=5, early_stopping=True)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

original_texts = []
correct_translations = []
model_translations = []

for sample in tqdm(random_test_samples):
    input_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
    correct_translation = tokenizer.decode(sample['labels'], skip_special_tokens=True)
    model_translation = generate_translation(sample)

    original_texts.append(input_text)
    correct_translations.append(correct_translation)
    model_translations.append(model_translation)
    
df_results = pd.DataFrame({
    "Original Text": original_texts,
    "Correct Translation": correct_translations,
    "Model Translation": model_translations
})

100%|██████████| 1/1 [00:17<00:00, 17.37s/it]

CPU times: user 9.32 s, sys: 1.08 s, total: 10.4 s
Wall time: 17.4 s


In [33]:
df_results.loc[:, 'Original Text'] = df_results['Original Text'].str.replace('__en__', '')
df_results.loc[:, 'Correct Translation'] = df_results['Correct Translation'].str.replace('__en__', '')
df_results.loc[:, 'Model Translation'] = df_results['Model Translation'].str.replace('__en__', '')
    
bleu_score = sacrebleu.corpus_bleu(df_results['Model Translation'].tolist(), 
                                   [df_results['Correct Translation'].tolist()]).score
chrf_score = sacrebleu.corpus_chrf(df_results['Model Translation'].tolist(), 
                                   [df_results['Correct Translation'].tolist()]).score

print(f"BLEU Score: {bleu_score}")
print(f"ChrF Score: {chrf_score}")

df_results.head(50)

BLEU Score: 17.566674560348577
ChrF Score: 45.26100148066748


,Original Text,Correct Translation,Model Translation
0,Йпыг-пы̄грись нампа о̄йка ань та̄н ли па̄ланы...,Мужчина по имени Йипыг-пыгрись жил севернее и...,Мужчина по имени Ипыг-пыгрись сейчас на их бе...
1,О̄с айыс.,Опять выпила.,Опять поймал.
2,"Хунь Иаков хлыстэ, Иосиф лылы, сыме хот-ся̄гтыс.","Когда Иаков услышал, что Иосиф жив, то возлик...","Когда Иаков услышал, что Иосиф жив, сердце ис..."
3,Ос илттыг арēнан са̄в акроба̄т та ха̄йтлыгтас...,И на арену выбежало множество акробатов...,И вдруг в арню много сразу побежало...
4,Фашистыт 872 хо̄тал Ленинградын нэ̄мхотьют ат...,"Фашисты 872 дня в Лениград никого впускали, г...","Фашисты 872 дня в Ленинград никто не пускали,..."
5,Этот старик рыбачит.,Ты матум о̄йка хл алыщлы.,Ты матум о̄йка хл алыщлы.
6,"Та̄л сыс ма̄хманэ ёт маныр ва̄рсыт, тав ань п...",В течение года что они с сотрудниками сделали...,"За год с людьми что делали, она сейчас разгов..."
7,Нор аняв акваг та яныгми.,Гора из бревен всё растёт.,Наше горное чаше постоянно растёт.
8,Э̄ква-пыгрись Па̄йпы э̄ква ня̄врамыг ма̄гыс м...,Что предложил Эква пыгрись смастерить детям ж...,Эква-пыгрись Пайпы женщина за детей что делат...
9,"Спросите вот моего сынишку, какого мы раз пой...","Китылэ̄н ам пы̄гум, ма̄нхурп ман акв сёс пвы̄...","Ам пыгрищум ты китыглэ̄н, манхурип порат ма̄н..."
